In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

### model.parameters() / model.module() / model.children()

In [2]:
class test_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU()
        )
        self.MaxPool1 = nn.MaxPool2d(2, 2)

        self.fc = nn.Sequential(
            nn.Linear(256, 64),
            nn.Linear(64, 10)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.MaxPool1(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

model = test_model()

x = torch.randn(32, 3, 8, 8)
print(model(x).shape)

torch.Size([32, 10])


- model.parameters()

In [3]:
model_params = list(model.parameters())
for pram in model_params:
    print(pram.shape)

model_name_parms = list(model.named_parameters())
for name, param in model_name_parms:
    print(name, param.shape)

torch.Size([16, 3, 3, 3])
torch.Size([16])
torch.Size([16])
torch.Size([16])
torch.Size([64, 256])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])
conv1.0.weight torch.Size([16, 3, 3, 3])
conv1.0.bias torch.Size([16])
conv1.1.weight torch.Size([16])
conv1.1.bias torch.Size([16])
fc.0.weight torch.Size([64, 256])
fc.0.bias torch.Size([64])
fc.1.weight torch.Size([10, 64])
fc.1.bias torch.Size([10])


- model.module()

In [4]:
model_modules = list(model.modules())

for module in model_modules:
    print(module, end='\n\n')

test_model(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (MaxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=10, bias=True)
  )
)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

ReLU()

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

Sequential(
  (0): Linear(in_features=256, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=10, bias=Tru

In [5]:
for module in model_modules:
    if isinstance(module, nn.Linear):
        print(module)

Linear(in_features=256, out_features=64, bias=True)
Linear(in_features=64, out_features=10, bias=True)


- model.children()

In [6]:
model_children = model.children()
for child in model_children:
    print(child)

Sequential(
  (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Sequential(
  (0): Linear(in_features=256, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=10, bias=True)
)


### nn.Sequential() vs nn.ModuleList()
- nn.Sequential()은 내부에 forward(self, x)가 이미 선언되어 있음
- nn.ModuleList()는 선언되어 있지 않아, 반복문으로 돌려주어야 한다.

In [7]:
class test_model_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.ModuleList([nn.Linear(3, 5), nn.Linear(5, 2)])
    
    def forward(self, x):
        for module in self.fc:
            x = module(x)
        return x

model = test_model_2()
x = torch.randn(32, 3)
print(model(x).shape)

torch.Size([32, 2])


- nn.Sequential()의 forward(self, x)는 하나의 인자 'x'만 입력으로 받을 수 있게 선언 되어 있음
- 따라서, 아래 그림과 같이 여러개의 input을 받을 땐 nn.ModuleList()로 묶어주어야 한다.
    - 아니면 overriding을 통해 Sequential()의 forward(self, x)를 overriding 해주면 되지만, 매우 번거로움

In [8]:
class two_input_block(nn.Module):
    def __init__(self):
        super().__init__()
        self.block_x = nn.Linear(1,1)
        self.block_y = nn.Linear(1,1)

    def forward(self, x, y):
        x = self.block_x(x)
        y = self.block_y(y)
        return x, y

x = torch.randn(1)
y = torch.randn(1)
block = two_input_block()


In [9]:
# Sequential()은 input을 1개밖에 못받아 에러 발생!!

model = nn.Sequential(block, block)
print(model)
# print(model(x, y)) # TypeError: Sequential.forward() takes 2 positional arguments but 3 were given

Sequential(
  (0): two_input_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
  (1): two_input_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
)


In [10]:
# ModuleList()는 정상적으로 동작
model = nn.ModuleList([block, block])
print(model)

for module in model:
    x, y = module(x, y)

print(x, y)

ModuleList(
  (0-1): 2 x two_input_block(
    (block_x): Linear(in_features=1, out_features=1, bias=True)
    (block_y): Linear(in_features=1, out_features=1, bias=True)
  )
)
tensor([0.9010], grad_fn=<ViewBackward0>) tensor([-0.7572], grad_fn=<ViewBackward0>)
